# MVR datasets merge

```markdown
Inputs: 
    - cleaned_countyoutflow.csv: outlfow migration data, 
    - zillow_yearly_clean.csv: housing data,
    - health_data_clean.csv: health data

outputs: 
    - merged dataset of shape (3209, 50)

In [1]:
import pandas as pd

## Part 1: Migration data

In [2]:

outflow = pd.read_csv('/Users/judithyemeli/Documents/CSE_6242/Project/MVR/Network_graph_analysis/cleaned_countyoutflow.csv')

In [3]:
outflow.head(3)

,year,origin_state_fips,origin_county_fips,destination_state_fips,destination_county_fips,y1fips,y2fips,y2_state,y2_countyname,num_returns,num_individuals,adjusted_gross_income
0,16-17,1,1,1,1,1001,1001,AL,Autauga County Non-migrants,17484,39711,1106647
1,16-17,1,21,1,1,1021,1001,AL,Autauga County,83,220,3006
2,16-17,1,47,1,1,1047,1001,AL,Autauga County,82,195,2917


``` markdown
From both datasets, the columns carrying information are returns, individuas and gross income.
Renaming those attributes is required to ensure they are distict after merging.

In [4]:
# rename the key attributes 
outflow_attributes_dict = {'num_returns': 'out_return', 'num_individuals': 'out_individuals', 'adjusted_gross_income': 'out_gross_income'}

In [5]:
outflow.rename(columns=outflow_attributes_dict, inplace=True)

``` markdown
Grouping each dataset to have a single line for each county. The aggregation method is sum across the ckey attributes idenfified earlier.

In [6]:
outflow= outflow.groupby(by= ['y2_state','destination_county_fips'])[['out_return', 'out_individuals', 'out_gross_income']].sum().reset_index()
outflow.head(3)

,y2_state,destination_county_fips,out_return,out_individuals,out_gross_income
0,AK,0,364248,733000,22577044
1,AK,1,48217,91547,2853137
2,AK,3,131234,269101,8294827


In [7]:
# print(f"lenght of grouped inflow: ",len(grouped_inflow))
print(f"lenght of grouped outflow: ",len(outflow))

lenght of grouped outflow:  3209


## Part 2: House pricing

``` markdown
- Clean up the county idenfiers
- Calcule the net change by county between  2000 and 2024 (net_housing_price_change)
- Extract the current house price as 2024 (2024_house_price)
- Merge with migration data

In [8]:
house_pricing = pd.read_csv('/Users/judithyemeli/Documents/CSE_6242/Project/MVR/Network_graph_analysis/Zillow_Data_Yearly_FIPS.csv')
house_pricing.head(3)

,FIPS,RegionName,State,FIPS.1,2000,2001,2002,2003,2004,2005,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,2020,Anchorage Borough,AK,2020,146144.3161,169973.1478,186475.8294,195969.8382,209724.6331,229868.8594,...,304919.1673,311721.6694,310289.3725,312758.8579,330962.7465,338964.0436,344423.7324,365491.1856,377498.0278,384018.9248
1,2090,Fairbanks North Star Borough,AK,2090,NaN,NaN,NaN,NaN,NaN,188322.2179,...,225121.4489,233876.6726,243537.0955,252751.3460,265862.9022,277602.7961,285834.8267,300548.1123,296752.7637,301036.6606
2,2100,Haines Borough,AK,2100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,284891.8249,315908.7940,298008.7765,294193.3742


Use the raw dataset that has all the counties

In [9]:
# t = pd.read_csv('/Users/judithyemeli/Documents/CSE_6242/Project/MVR/Network_graph_analysis/zillow_yearly_clean.csv')
# len(t)

In [10]:
len(house_pricing)

3074

In [11]:
house_pricing.isna().sum()

FIPS             0
RegionName       0
State            0
FIPS.1           0
2000          2020
2001          1979
2002          1938
2003          1880
2004          1837
2005          1788
2006          1758
2007          1704
2008          1634
2009          1059
2010           985
2011           921
2012           647
2013           597
2014           560
2015           529
2016           101
2017            94
2018            86
2019            68
2020            63
2021            51
2022            12
2023             2
2024             0
dtype: int64

In [12]:
cols_to_fillna = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024'] 
n = int(len(cols_to_fillna)/2) # first half of the columns are years_min and the second half are years_max
cols_years_min = cols_to_fillna[:n]
cols_years_max = cols_to_fillna[n:]


In [13]:
house_pricing[cols_to_fillna] = house_pricing[cols_to_fillna].fillna(house_pricing[cols_to_fillna].mean())  
house_pricing.isna().sum()

FIPS          0
RegionName    0
State         0
FIPS.1        0
2000          0
2001          0
2002          0
2003          0
2004          0
2005          0
2006          0
2007          0
2008          0
2009          0
2010          0
2011          0
2012          0
2013          0
2014          0
2015          0
2016          0
2017          0
2018          0
2019          0
2020          0
2021          0
2022          0
2023          0
2024          0
dtype: int64

In [14]:
# defining a function to format the house_pricing data to be the same as the merged migration 
def county_code_reformat(value):
        # ensure the value is a string
        num_str = str(value)
        num_str = num_str[-3:]
            # check if the new first character is zero
        if num_str[0] == '0':
            # remove the zero
            num_str = num_str[1:]
            # repeat
            if num_str[0] == '0':
                num_str = num_str[1:]
                return int(num_str)
            else:
                 return int(num_str)
        else:
             return int(num_str)

In [15]:
house_pricing['short_county_code'] = house_pricing['FIPS'].apply(county_code_reformat) 
null_values = house_pricing['short_county_code'].isnull().sum()
print(f"Number of null values in short_county_code column: {null_values}")

Number of null values in short_county_code column: 0


In [16]:
# cast the typpe for the short_county_code as integer
house_pricing['short_county_code'] = house_pricing['short_county_code'].astype(int)
house_pricing.isna().sum()


FIPS                 0
RegionName           0
State                0
FIPS.1               0
2000                 0
2001                 0
2002                 0
2003                 0
2004                 0
2005                 0
2006                 0
2007                 0
2008                 0
2009                 0
2010                 0
2011                 0
2012                 0
2013                 0
2014                 0
2015                 0
2016                 0
2017                 0
2018                 0
2019                 0
2020                 0
2021                 0
2022                 0
2023                 0
2024                 0
short_county_code    0
dtype: int64

In [17]:
house_pricing.drop(columns=['RegionName', 'FIPS.1'], inplace=True)

In [18]:
house_pricing = house_pricing.groupby(['State', 'FIPS']).mean().reset_index()
len(house_pricing)

3074

In [19]:
# house_pricing_cleaned = house_pricing[['State','short_county_code', 'house_index', 'housing_avg_value']]
full_dataset = outflow.merge(house_pricing, left_on=['y2_state', 'destination_county_fips'], right_on=['State', 'short_county_code'], how='left')
print(len(full_dataset))
full_dataset.isna().sum()

3209


y2_state                     0
destination_county_fips      0
out_return                   0
out_individuals              0
out_gross_income             0
State                      135
FIPS                       135
2000                       135
2001                       135
2002                       135
2003                       135
2004                       135
2005                       135
2006                       135
2007                       135
2008                       135
2009                       135
2010                       135
2011                       135
2012                       135
2013                       135
2014                       135
2015                       135
2016                       135
2017                       135
2018                       135
2019                       135
2020                       135
2021                       135
2022                       135
2023                       135
2024                       135
short_co

## Health data

In [20]:
# health = pd.read_csv('/Users/judithyemeli/Documents/CSE_6242/Project/MVR/Network_graph_analysis/Health_Data.csv')

In [21]:
health = pd.read_csv('/Users/judithyemeli/Documents/CSE_6242/Project/MVR/Network_graph_analysis/health_data_clean.csv')
health.head(3)

,Year,FIPS,State,County,X..Fair.Poor,Physically.Unhealthy.Days,Mentally.Unhealthy.Days,X..Smokers,X..Obese,X..Physically.Inactive,X..Excessive.Drinking,X..Some.College,Population.1,X..Some.College.1,X..Social.Associations,Association.Rate,X..Severe.Housing.Problems,X..Insufficient.Sleep
0,2017,1001,Alabama,Autauga,18,4.2,4.2,17,34,29,15,8617,14440,59.7,76,13.7,15,38
1,2017,1003,Alabama,Baldwin,16,3.7,4.0,18,27,22,15,29788,47367,62.9,231,11.5,15,33
2,2017,1005,Alabama,Barbour,25,4.8,4.8,23,44,32,12,2839,7230,39.3,22,8.2,16,40


In [22]:
health['short_county_code'] = health['FIPS'].apply(county_code_reformat)    

In [23]:
state_to_abbreviation = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", 
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA",
    "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA",
    "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT",
    "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
    "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
}

In [24]:
health['State_short'] = health['State'].map(state_to_abbreviation)
health['State_short'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', nan, 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [25]:
health.drop(columns=['State', 'County', 'FIPS'], inplace=True)
health.rename(columns={'State_short': 'State', 'short_county_code':'FIPS'}, inplace=True)

In [26]:
health.head(3)

,Year,X..Fair.Poor,Physically.Unhealthy.Days,Mentally.Unhealthy.Days,X..Smokers,X..Obese,X..Physically.Inactive,X..Excessive.Drinking,X..Some.College,Population.1,X..Some.College.1,X..Social.Associations,Association.Rate,X..Severe.Housing.Problems,X..Insufficient.Sleep,FIPS,State
0,2017,18,4.2,4.2,17,34,29,15,8617,14440,59.7,76,13.7,15,38,1,AL
1,2017,16,3.7,4.0,18,27,22,15,29788,47367,62.9,231,11.5,15,33,3,AL
2,2017,25,4.8,4.8,23,44,32,12,2839,7230,39.3,22,8.2,16,40,5,AL


In [27]:
health_grouped = health.groupby(by=['State', 'FIPS']).mean().reset_index()
len(health_grouped)

3142

In [28]:
health_grouped.head(3)

,State,FIPS,Year,X..Fair.Poor,Physically.Unhealthy.Days,Mentally.Unhealthy.Days,X..Smokers,X..Obese,X..Physically.Inactive,X..Excessive.Drinking,X..Some.College,Population.1,X..Some.College.1,X..Social.Associations,Association.Rate,X..Severe.Housing.Problems,X..Insufficient.Sleep
0,AK,13,2019.0,17.6,3.62,2.94,18.2,42.2,26.6,16.8,434.8,1005.6,43.20,1.6,4.80,13.0,143.0
1,AK,16,2019.0,15.6,3.22,2.78,16.2,41.2,25.8,18.6,987.8,1892.8,52.24,2.8,4.92,19.0,94.6
2,AK,20,2019.0,13.6,3.48,3.32,15.4,29.2,17.4,20.0,63400.2,89256.8,70.96,339.4,11.44,16.6,13097.8


In [29]:
health_grouped.isna().sum()

State                         0
FIPS                          0
Year                          0
X..Fair.Poor                  0
Physically.Unhealthy.Days     0
Mentally.Unhealthy.Days       0
X..Smokers                    0
X..Obese                      0
X..Physically.Inactive        0
X..Excessive.Drinking         0
X..Some.College               0
Population.1                  0
X..Some.College.1             0
X..Social.Associations        0
Association.Rate              0
X..Severe.Housing.Problems    0
X..Insufficient.Sleep         0
dtype: int64

In [30]:
full_dataset.head(3)

,y2_state,destination_county_fips,out_return,out_individuals,out_gross_income,State,FIPS,2000,2001,2002,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,short_county_code
0,AK,0,364248,733000,22577044,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,1,48217,91547,2853137,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,3,131234,269101,8294827,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
full_dataset = full_dataset.merge(health_grouped, left_on=['y2_state', 'destination_county_fips'], right_on=['State', 'FIPS'], how='left')
len(full_dataset)

3209

In [32]:
full_dataset.shape

(3209, 50)

In [33]:
full_dataset.to_csv('full_dataset.csv', index=False)